Variable Scope: LEGB Rule
1. Local: Defined inside current function.
2. Enclosed: Defined in outer function (nested).
3. Global: Defined at the top level of the script.
4. Built-in: Reserved keywords like range.

Keywords to specify scope of variables:
1. `nonlocal var_name`: To modify an enclosed variable.
2. `global var_name`: To modify a global variable inside a function.

### Closure:
- Occurs when a nested function remembers arguments passed in the outer function even after the latter has finished execution.

In [3]:
def add_n(n):

    def add_to_x(x):  # The nested function remembers the value of n from the outer function
        return x+n

    return add_to_x  # The outer function returns the inner function object that can be modified and duplicated.

add_5 = add_n(5)
add_10 = add_n(10)

print(add_5(44))
print(add_10(20))

49
30


In [18]:
def make_counter():
    c = 0

    def add_to():
        nonlocal c
        c += 1
        return c

    return add_to

counter = make_counter()
print(counter()) # Should print 1
print(counter()) # Should print 2

1
2


### Decorators:
- A function that takes another function and wraps it to extend its behaviour.
- Doesn't explicitly modify its source code.

- Instead of writing decorated_func = decorator(original_func), Python uses the @ symbol as "syntactic sugar".

In [25]:
def dec_1(func):

    def wrapper(*args, **kwargs):
        print("Top Layer")
        print(func(*args, **kwargs))
        print("Bottom Layer")

    return wrapper

@dec_1
def add(a, b):
    return a + b

add(5,10)

Top Layer
15
Bottom Layer


In [31]:
from time import perf_counter, sleep

def timer(func):

    def wrapper(*args, **kwargs):
        start = perf_counter()
        suum = func(*args, **kwargs)
        end = perf_counter()
        print(f"Execution time = {end - start} s")
        return suum

    return wrapper

@timer
def add(a, b):
    return a + b

print(add(28348, 458493))

@timer
def waste_time():
    sleep(1) # Pauses for 1 second
    print("Finished sleeping!")

waste_time()


Execution time = 6.999998731771484e-07 s
486841
Finished sleeping!
Execution time = 1.0009824000007939 s


In [33]:
def singleton(cls):
    instances = {}

    def wrapper(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)

        return instances[cls]

    return wrapper

In [34]:
def require_login(func):
    def wrapper(user, *args, **kwargs):
        if user.is_authenticated:
            func(user, *args, **kwargs)
        else:
            raise PermissionError("User is not authenticated.")

    return wrapper


In [37]:
def limit_calls(n):
    def decorator(func):

        call_limit = n

        def check_lim(*args, **kwargs):
            nonlocal call_limit

            if call_limit:
                call_limit -= 1
                return func(*args, **kwargs)
            else:
                print("Error: Call limit reached")

        return check_lim

    return decorator

@limit_calls(3)
def say_hello():
    print("Hello!")

say_hello()
say_hello()
say_hello()
say_hello()

Hello!
Hello!
Hello!
Error: Call limit reached


In [39]:
history = []
current_value = 0

def add(val):
    global current_value
    current_value += val

def multiply(val):
    global current_value
    current_value *= val

def execute(func, val):
    global current_value
    # 1. Perform the action
    func(val)
    print(f"Current Value: {current_value}")

    # 2. Store the "Inverse" in a lambda
    if func == add:
        # We capture 'val' inside this lambda's closure!
        history.append(lambda: add(-val))
    elif func == multiply:
        history.append(lambda: multiply(1/val))

def undo():
    if history:
        undo_func = history.pop()
        undo_func()
        print(f"Undo performed! Current Value: {current_value}")
    else:
        print("Nothing to undo.")

execute(add, 10)      # Value: 10
execute(multiply, 4)   # Value: 40
undo()

Current Value: 10
Current Value: 40
Undo performed! Current Value: 10.0
